In [1]:
import pandas as pd 
import numpy as np
import pyecharts
import plotly.express as px # visualization
import pandas_datareader as pdr # access fred
from datetime import date
from fredapi import Fred
import requests
import openpyxl
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
import re

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

from pyecharts import options as opts
from pyecharts.charts import Bar, Line

In [328]:
# Load orginal dataset

index_data = pd.read_excel('C:/Users/fanjz/Desktop/dataset/pmi/pmi_level.xlsx')
glance = pd.read_excel('C:/Users/fanjz/Desktop/dataset/pmi/glancetable.xlsx')
comments = pd.read_excel('C:/Users/fanjz/Desktop/dataset/pmi/comments.xlsx')
sector_ranking = pd.read_excel('C:/Users/fanjz/Desktop/dataset/pmi/sector_ranking.xlsx')

In [329]:
# Copy and paste latest month data url

url = input('Please Paste Latest PMI URL: ')

url_str = str(url)

response = requests.get(url_str)

### Extract Data from URL ###

#### 1. Get comments from respondents ####

In [330]:
# Extract What Respondents Say

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        main_content = section.find('div', class_='col-lg-10 col-lg-offset-1')

        if main_content:
            ul_element = main_content.find('ul', type='disc')
            if ul_element:
                list_items = ul_element.find_all('li')
                if list_items:
                    # Create a list to store the text of each list item
                    items_list = [item.text.strip() for item in list_items]

                    # Create a DataFrame from the list
                    respondent_comment = pd.DataFrame(items_list, columns=['WHAT RESPONDENTS ARE SAYING'])
        else:
            print("Main content not found within the section.")
    else:
        print("Section with class 'xxxx' not found.")

else:
    print("Failed to retrieve the page")

#### 2. Get Monthly Level Data ####

In [331]:
# Extract monthly index level

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')   
    section = soup.find('section', class_='release-body container')

    if section:
        table_div = section.find('div', class_='table-responsive')

        if table_div:
        # Find the table within the div
            table = table_div.find('table')

            if table:
                # Read the table into a DataFrame
                data_table = pd.read_html(str(table))[0]
            else:
                print("Table not found within the div with class 'table-responsive'")
        else:
            print("Div with class 'table-responsive' not found.")
    else:
        print("Failed to retrieve the page")

C:\Users\fanjz\AppData\Local\Temp\ipykernel_4388\2319469250.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data_table = pd.read_html(str(table))[0]


#### 3. Get Commodity Data ####

In [334]:
# Extract Commodity up/down/short supply

commodity_up = soup.select_one('#main > article > section > div > div > p:nth-child(15)')
commodity_down = soup.select_one('#main > article > section > div > div > p:nth-child(16)')
commodity_short = soup.select_one('#main > article > section > div > div > p:nth-child(17)')

commodity_tags = [commodity_up,commodity_down,commodity_short]
commoditiy_direction = []
for i, tag in enumerate(commodity_tags):
    if tag:
        direction = tag.get_text(separator=' ').strip()  # Get all the text within the <p> tag
        commoditiy_direction.append(direction)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
commodity_comments = pd.DataFrame(commoditiy_direction, columns=['content'])

In [335]:
substring_patterns = ['Commodities Up in Price','Commodities Down in Price','Commodities in']

def split_based_on_pattern(content, pattern):
    if pattern in content:
        parts = content.split(pattern, 1)
        return [pattern.strip(), parts[1].strip()]
    else:
        return [content.strip(), '']

# Split each row based on the defined patterns
split_data = []
for i, row in commodity_comments.iterrows():
    split_data.append(split_based_on_pattern(row['content'], substring_patterns[i]))

# Create a new DataFrame from the split data
commodity_df = pd.DataFrame(split_data, columns=['pattern', 'rest'])

# Display the split DataFrame
commodity_df

,pattern,rest
0,Commodities Up in Price,Aluminum (5); Copper; Corrugated Boxes (2); Corrugated Sheets (2); Crude Oil (2); Diesel; Gasoline (2); High-Density Polyethylene (HDPE) Resin; Pl...
1,Commodities Down in Price,Steel — Carbon*.
2,Commodities in,Short Supply Electrical Components (43); Electrical Equipment (3); Electronic Components; and Labor — Temporary.


#### 4. Get Industry Ranking #### 

In [344]:
# # Extract Industry Rankings

pmi = soup.select_one('#main > article > section > div > div > p:nth-child(9)')
new_order = soup.select_one('#main > article > section > div > div > p:nth-child(25)')
production = soup.select_one('#main > article > section > div > div > p:nth-child(28)')
employment= soup.select_one('#main > article > section > div > div > p:nth-child(31)')
supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(34)')
inventories = soup.select_one('#main > article > section > div > div > p:nth-child(37)')
customers_inv = soup.select_one('#main > article > section > div > div > p:nth-child(40)')
prices = soup.select_one('#main > article > section > div > div > p:nth-child(43)')
backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(46)')
new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(49)')
imports = soup.select_one('#main > article > section > div > div > p:nth-child(52)')

# # Extract Industry Rankings

# pmi = soup.select_one('#main > article > section > div > div > p:nth-child(10)')
# new_order = soup.select_one('#main > article > section > div > div > p:nth-child(26)')
# production = soup.select_one('#main > article > section > div > div > p:nth-child(29)')
# employment= soup.select_one('#main > article > section > div > div > p:nth-child(32)')
# supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(35)')
# inventories = soup.select_one('#main > article > section > div > div > p:nth-child(38)')
# customers_inv = soup.select_one('#main > article > section > div > div > p:nth-child(41)')
# prices = soup.select_one('#main > article > section > div > div > p:nth-child(44)')
# backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(47)')
# new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(50)')
# imports = soup.select_one('#main > article > section > div > div > p:nth-child(53)')

# Extract Industry Rankings

# pmi = soup.select_one('#main > article > section > div > div > p:nth-child(11)')
# new_order = soup.select_one('#main > article > section > div > div > p:nth-child(27)')
# production = soup.select_one('#main > article > section > div > div > p:nth-child(30)')
# employment= soup.select_one('#main > article > section > div > div > p:nth-child(33)')
# supplier_deliveries = soup.select_one('#main > article > section > div > div > p:nth-child(36)')
# inventories = soup.select_one('#main > article > section > div > div > p:nth-child(39)')
# customers_inv = soup.select_one('#main > article > section > div > div > p:nth-child(42)')
# prices = soup.select_one('#main > article > section > div > div > p:nth-child(45)')
# backlog_orders = soup.select_one('#main > article > section > div > div > p:nth-child(48)')
# new_export_orders = soup.select_one('#main > article > section > div > div > p:nth-child(51)')
# imports = soup.select_one('#main > article > section > div > div > p:nth-child(54)')



In [345]:
industry_ranking = [pmi,new_order,production,employment,supplier_deliveries,inventories,customers_inv,prices,backlog_orders,new_export_orders,imports]

rankings = []
for i, tag in enumerate(industry_ranking):
    if tag:
        ranking = tag.get_text().strip()  # Get all the text within the <p> tag
        rankings.append(ranking)
    else:
        print(f"The specified <p> tag {i+1} was not found.")

# Create a DataFrame with one column and three rows from the list of contents
ranking_df = pd.DataFrame(rankings, columns=['Ranking'])
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip()


In [348]:
industry_list = ['PMI','New Order','Production','Employment','Supplier Deliveries','Inventories','Customers_inv','Prices','Backlog Orders','New Export Orders','Imports']
ranking_df['Industry'] = industry_list
col_oder = ['Industry','Ranking']
ranking_df = ranking_df[col_oder]

In [349]:
ranking_df

,Industry,Ranking
0,PMI,The nine manufacturing industries reporting growth in April — in order — are: Nonmetallic Mineral Products; Printing & Related Support Activities;...
1,New Order,The eight manufacturing industries that reported growth in new orders in April — in the following order — are: Printing & Related Support Activiti...
2,Production,"The eight industries reporting growth in production during the month of April, in order, are: Plastics & Rubber Products; Petroleum & Coal Product..."
3,Employment,"Of 18 manufacturing industries, the four industries reporting employment growth in April are: Textile Mills; Nonmetallic Mineral Products; Transpo..."
4,Supplier Deliveries,"The four manufacturing industries reporting slower supplier deliveries in April are: Nonmetallic Mineral Products; Primary Metals; Food, Beverage ..."
5,Inventories,"Of 18 manufacturing industries, four reported higher inventories in April: Wood Products; Electrical Equipment, Appliances & Components; Nonmetall..."
6,Customers_inv,"The five industries reporting customers' inventories as too high in April are: Textile Mills; Wood Products; Food, Beverage & Tobacco Products; Mi..."
7,Prices,"In April, the 13 industries that reported paying increased prices for raw materials, in order, are: Printing & Related Support Activities; Textile..."
8,Backlog Orders,"Of 18 manufacturing industries, the three that reported growth in order backlogs in April are: Wood Products; Nonmetallic Mineral Products; and Pl..."
9,New Export Orders,The five industries reporting growth in new export orders in April are: Wood Products; Plastics & Rubber Products; Miscellaneous Manufacturing; Fo...


### Working on Data Clenaing ### 

##### 1. PMI Index Level #####

In [350]:
# variables = data_table, respondent_comment, commodity_df, ranking_df
# 1. Working on data_table

new_table = data_table.iloc[1:]
header = new_table.iloc[0]
final_table = new_table.iloc[1:]
final_table.columns = header

In [351]:
new_month_level = final_table.iloc[:,1]
new_month_level = new_month_level[:-2]
new_month_level = new_month_level.to_frame().T

In [352]:
latest_date = index_data['Date'].max()
new_date = latest_date + relativedelta(months=1)
new_date_str = new_date.strftime('%Y-%m-%d')
new_month_level["Date"] = new_date
new_month_level = new_month_level[['Date'] + [col for col in new_month_level if col != 'Date']]
new_month_level.columns = index_data.columns

updated_index_data = pd.concat([index_data, new_month_level], ignore_index=True)

##### 2. What Respondents Saying #####

In [353]:
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 150)
respondent_comment

,WHAT RESPONDENTS ARE SAYING
0,"""Conditions are improving as demand is starting to recover. Costs continue to be a major concern as suppliers that rapidly increased prices in the..."
1,"""Sales continue to exceed expectations in 2024. The forecasted dip in commercial vehicle production volumes appears to be avoided. Operational out..."
2,"""Order flow has stabilized. It took some customers longer to replenish their supply chain network after the fourth-quarter rush we commonly have. ..."
3,"""Some small indications of market improvement in China for our instruments and technology. Recovery is still slower than we had hoped, and macroec..."
4,"""Market conditions have definitely softened. Thankfully, our backlog is strong and will get us through the year. When conditions improve as expect..."
5,"""Business is slowing down — it has been a gradual decline for the last several months. We are not seeing new orders at last year's level, or at th..."
6,"""There has been a lot of volatility in sales. On average, our sales look flat, but the volatility is concerning."" [Electrical Equipment, Appliance..."
7,"""Business remained strong through the first quarter and has started strong for the second quarter. Commercial construction is still going well but..."
8,"""The major factor affecting our business is the uncertainty of the Federal Reserve's handling of interest rates, which will affect our customers' ..."
9,"""Business is stable, and orders have been consistent. We're quoting new business for the factory, and automotive builds continue at averages but n..."


In [354]:
updated_comments = respondent_comment['WHAT RESPONDENTS ARE SAYING'].str.split('[', expand=True)
updated_comments.columns = [new_date_str, 'Industry']
updated_comments['Industry'] = updated_comments['Industry'].str.replace(']', '', regex=False)
updated_comments[new_date_str] = updated_comments[new_date_str].str.replace('"', '', regex=False)
updated_comments = updated_comments.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

col_order = ['Industry',new_date_str]
updated_comments = updated_comments[col_order]

In [355]:
industry_col1 = comments['Industry']
industry_col2 = updated_comments['Industry']

merged_df = pd.concat([industry_col1, industry_col2], ignore_index=True)
merged_series = merged_df.squeeze().drop_duplicates()
merge_comment = merged_series.to_frame(name='Industry')

In [356]:
merge_final_df1 = pd.merge(merge_comment, comments, on='Industry', how='left')
final_comment = pd.merge(merge_final_df1, updated_comments, on='Industry', how='left')

##### 3. Commodity Product Pricing #####

In [357]:
commodity_df

commodity_dfs = []

for index, row in commodity_df.iterrows():
    # Create a new DataFrame for the current row
    row_df = pd.DataFrame([row])
    # Append the new DataFrame to the list
    commodity_dfs.append(row_df)
    

In [358]:
commodity_up_df = commodity_dfs[0]
commodity_up_df_split = commodity_up_df['rest'].str.split(';', expand = True).T
commodity_up_header = ['Commodities Up in Price']
commodity_up_df_split.columns = commodity_up_header


commodity_down_df = commodity_dfs[1]
commodity_down_df_split = commodity_down_df['rest'].str.split(';', expand = True).T
commodity_down_header = ['Commodities Down in Price']
commodity_down_df_split.columns = commodity_down_header

commodity_short_df = commodity_dfs[2]
commodity_short_df_split = commodity_short_df['rest'].str.split(';', expand = True).T
commodity_short_header = ['Commodities in Short Supply']
commodity_short_df_split.columns = commodity_short_header

commodity_up_df

In [359]:
commodity_up_df_split[['Commodity Price Up', 'Number of Month']] = commodity_up_df_split['Commodities Up in Price'].str.split("(", n=1, expand=True)
commodity_up_df_split['Commodity Price Up'] = commodity_up_df_split['Commodity Price Up'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_up_df_split['Number of Month'] = commodity_up_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
commodity_up_df_split = commodity_up_df_split.drop(columns=['Commodities Up in Price'])

commodity_down_df

In [ ]:
# commodity_down_df_split[['Commodity Price Down', 'Number of Month']] = commodity_down_df_split['Commodities Down in Price'].str.split("(", n=1, expand=True)
# commodity_down_df_split['Commodity Price Down'] = commodity_down_df_split['Commodity Price Down'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
# commodity_down_df_split['Number of Month'] = commodity_down_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
# commodity_down_df_split = commodity_down_df_split.drop(columns=['Commodities Down in Price'])

In [361]:
if '(' in commodity_down_df_split:
    commodity_down_df_split[['Commodity Price Down', 'Number of Month']] = commodity_down_df_split['Commodities Down in Price'].str.split("(", n=1, expand=True)
    commodity_down_df_split['Commodity Price Down'] = commodity_down_df_split['Commodity Price Down'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
    commodity_down_df_split['Number of Month'] = commodity_down_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
    commodity_down_df_split = commodity_down_df_split.drop(columns=['Commodities Down in Price'])
else:
    commodity_down_df_split['Number of Month'] = [1]
    commodity_down_df_split.rename(columns={'Commodities Down in Price': 'Commodity Price Down'})

Commodity in short supply

In [362]:
commodity_short_df_split[['Commodities: Short Supply', 'Number of Month']] = commodity_short_df_split['Commodities in Short Supply'].str.split("(", n=1, expand=True)
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.strip().str.replace(r'\band\b|\*|\.', '', regex=True)
commodity_short_df_split['Number of Month'] = commodity_short_df_split['Number of Month'].str.rstrip(")").str.rstrip(").").fillna('1')
commodity_short_df_split = commodity_short_df_split.drop(columns=['Commodities in Short Supply'])
commodity_short_df_split['Commodities: Short Supply'] = commodity_short_df_split['Commodities: Short Supply'].str.replace('Short Supply ', '', regex=True)

Commodity Data Final Output

In [363]:
commodity_short_df_split = commodity_short_df_split.sort_values(['Number of Month'])
commodity_up_df_split = commodity_up_df_split.sort_values(['Number of Month'])
commodity_down_df_split = commodity_down_df_split.sort_values(['Number of Month'])

##### 4. Industry Ranking #####

In [364]:
ranking_df['Ranking'] = ranking_df['Ranking'].str.strip().str.replace('—','',regex = False).str.replace(',','',regex = False)
ranking_df_split = ranking_df['Ranking'].str.split('.', expand = True)
ranking_df_split.columns = ['Industry Up','Industry Down','No Change','None']

In [365]:
ranking_up_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Up']], axis=1)
ranking_down_df = pd.concat([ranking_df['Industry'], ranking_df_split['Industry Down']], axis=1)

In [366]:
ranking_up = ranking_up_df['Industry Up'].str.split(':', expand=True)
ranking_down = ranking_down_df['Industry Down'].str.split(':', expand=True)

col_name = ['split_col','ranking_col']
ranking_up.columns = col_name
ranking_down.columns = col_name

In [367]:
filter_is_up = ranking_up['split_col'].str.split('is ', expand=True)
filter_is_down = ranking_down['split_col'].str.split('is ', expand=True)

In [368]:
filter_is_up = pd.concat([ranking_df['Industry'], filter_is_up], axis=1)
filter_is_down = pd.concat([ranking_df['Industry'], filter_is_down], axis=1)

In [369]:
def select_columns(df):
    if len(df.columns) >= 3:  # Check if DataFrame has at least 3 columns
        return df.iloc[:, [0, 2]]  # Select first and third columns
    else:
        return df.iloc[:, [0]]  # Select only the first column

filter_is_up_df = select_columns(filter_is_up)
filter_is_down_df = select_columns(filter_is_down)

In [370]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up['ranking_col']], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down['ranking_col']], axis=1)

In [371]:
ranking_up_final = pd.merge(ranking_up_final,filter_is_up_df, on= 'Industry')
ranking_down_final = pd.merge(ranking_down_final,filter_is_down_df, on= 'Industry')

In [372]:
def merge_columns(df):
    if len(df.columns) >= 3:
        df['Combined'] = df.iloc[:, 1].fillna(df.iloc[:, 2])
        df.drop(['ranking_col', 1], axis=1, inplace=True)
        return df
    else:
        df.rename(columns={'ranking_col': 'Combined'}, inplace=True)
        return df

ranking_up_final = merge_columns(ranking_up_final)
ranking_down_final = merge_columns(ranking_down_final)

In [373]:
ranking_up_final = ranking_up_final['Combined'].str.split(';', expand=True)
ranking_down_final = ranking_down_final['Combined'].str.split(';', expand=True)

In [374]:
ranking_up_final = ranking_up_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)
ranking_down_final = ranking_down_final.map(lambda x: x.replace('and', '').strip() if isinstance(x, str) else x)

In [375]:
ranking_up_final = pd.concat([ranking_df['Industry'], ranking_up_final], axis=1)
ranking_down_final = pd.concat([ranking_df['Industry'], ranking_down_final], axis=1)

In [376]:
rakning_up_column_order = [ranking_up_final.columns[0]] + list(range(1, len(ranking_up_final.columns)))
rakning_down_column_order = [ranking_down_final.columns[0]] + list(range(-1, -len(ranking_down_final.columns), -1))

ranking_up_final.columns = rakning_up_column_order
ranking_down_final.columns = rakning_down_column_order

In [377]:
ranking_up_df = pd.melt(ranking_up_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')
ranking_down_df = pd.melt(ranking_down_final, id_vars=['Industry'],var_name='myVarname', value_name='myValname')

In [378]:
industry_ranking_final = pd.concat([ranking_up_df, ranking_down_df],ignore_index=True)

industry_ranking_final = industry_ranking_final.rename(columns={'Industry': 'Index','myVarname': new_date_str, 'myValname': 'Industry'})

new_col_order = ['Index','Industry',new_date_str]
industry_ranking_final = industry_ranking_final[new_col_order]


In [379]:
desired_order = ranking_df['Industry'].to_list()

industry_ranking_final['Index'] = pd.Categorical(industry_ranking_final['Index'], categories=desired_order, ordered=True)
industry_ranking_final = industry_ranking_final.sort_values(['Index',new_date_str])
latest_industry_ranking = industry_ranking_final.dropna(subset=['Industry'])

sector_ranking_final = pd.merge(sector_ranking, latest_industry_ranking, on=['Index', 'Industry'], how='left')

# Final Step - Consolidation & Exporting  #

In [380]:
final_table

1,Index,Series Index Apr,Series Index Mar,Percentage Point Change,Direction,Rate of Change,Trend* (Months)
2,Manufacturing PMI®,49.2,50.3,-1.1,Contracting,From Growing,1
3,New Orders,49.1,51.4,-2.3,Contracting,From Growing,1
4,Production,51.3,54.6,-3.3,Growing,Slower,2
5,Employment,48.6,47.4,+1.2,Contracting,Slower,7
6,Supplier Deliveries,48.9,49.9,-1.0,Faster,Faster,2
7,Inventories,48.2,48.2,0.0,Contracting,Same,15
8,Customers' Inventories,47.8,44.0,+3.8,Too Low,Slower,5
9,Prices,60.9,55.8,+5.1,Increasing,Faster,4
10,Backlog of Orders,45.4,46.3,-0.9,Contracting,Faster,19
11,New Export Orders,48.7,51.6,-2.9,Contracting,From Growing,1


In [381]:
commodity_up_df_split

,Commodity Price Up,Number of Month
17,Zinc,1
1,Copper,1
5,Diesel,1
15,Steel Products,1
16,Titanium Dioxide,1
14,Steel — Hot Rolled,1
10,Precious Metals,1
11,Solvents,1
13,Steel — Carbon,1
12,Steel,10


In [382]:
commodity_down_df_split

,Commodities Down in Price,Number of Month
0,Steel — Carbon*.,1


In [323]:
commodity_short_df_split

,Commodities: Short Supply,Number of Month
2,Hydraulic Components,1
3,Plastic Resins,1
4,Semiconductors,1
1,Electrical Equipment,2
0,Electrical Components,42


In [383]:
updated_index_data.tail(5)

,Date,PMI,New Orders,Production,Employment,Deliveries,Inventories,Custom Inv,Prices,Ord Backlog,Exports,Imports
911,2023-12-01,47.4,47.1,50.3,48.1,47.0,44.3,48.1,45.2,45.3,49.9,46.4
912,2024-01-01,49.1,52.5,50.4,47.1,49.1,46.2,43.7,52.9,44.7,45.2,50.1
913,2024-02-01,47.8,49.2,48.4,45.9,50.1,45.3,45.8,52.5,46.3,51.6,53.0
914,2024-03-01,50.3,51.4,54.6,47.4,49.9,48.2,44.0,55.8,46.3,51.6,53.0
915,2024-04-01,49.2,49.1,51.3,48.6,48.9,48.2,47.8,60.9,45.4,48.7,51.9


In [384]:
final_comment.tail(5)

,Industry,2023-08-01 00:00:00,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01
12,"Apparel, Leather & Allied Products",NaN,"Markets remain soft. Our customers have about-right inventory levels, but they paid more due to pandemic cost increases. Everyone is holding off o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Nonmetallic Mineral Products,NaN,"Cement negotiations have changed, with cement mills no longer offering annual or guaranteed pricing. We now want to contract more as a commodity, ...","Commercial constructions continue to remain ahead of 2022. We have some concern over 2024 regarding inflation, as well as gas and oil pricing pote...","(Our situation is) good but guarded, as next year is hard to predict. There are undertones of uncertainty in the market and the impact of inflatio...",NaN,NaN,"We reflected on 2023 for maybe a minute and turned the page forward to 2024. Weather in January caused several operations to be idle, and shipment...",NaN,Business remained strong through the first quarter and has started strong for the second quarter. Commercial construction is still going well but ...
14,Petroleum & Coal Products,NaN,"A recession feels imminent. Money continues to be pushed into the bank markets, driving inflation rates really high. Most plants are buying less m...",Demand for raw materials/chemicals appears to be stable heading into the fourth quarter.,NaN,NaN,NaN,NaN,The potential aftermaths of the presidential election are beginning to impact conversations and negotiations of long-term agreements/contracts.,NaN
15,Textile Mills,NaN,"New business development is coming onboard. However, many forecasts are set for the beginning of 2024. Hiring and retaining quality people is stil...",NaN,NaN,NaN,"Remarkable slowdown in business in December. January has picked up, but not to previous-year levels.",NaN,NaN,NaN
16,Furniture & Related Products,NaN,NaN,"Business is decent — not great, but steady and solid. We are meeting our sales and margin goals, but it's definitely hard to guess the future.",NaN,NaN,NaN,NaN,NaN,NaN


In [385]:
sector_ranking_final.tail(5)

,ID,Index,Industry,Translation,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01,2023-10-01,2023-11-01,2023-12-01,2024-01-01,2024-02-01,2024-03-01,2024-04-01
193,11,Imports,Primary Metals,初级金属,3.0,NaN,1.0,2.0,-4.0,-6.0,2.0,NaN,-3.0,NaN,-1
194,11,Imports,Printing & Related Support Activities,印刷,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
195,11,Imports,Textile Mills,纺织业,NaN,1.0,NaN,1.0,-1.0,-2.0,NaN,NaN,NaN,NaN,NaN
196,11,Imports,Transportation Equipment,交通运输设备,-6.0,3.0,2.0,3.0,-9.0,3.0,-10.0,6.0,3.0,6.0,5
197,11,Imports,Wood Products,木材,-2.0,-1.0,-1.0,NaN,1.0,-1.0,-1.0,-1.0,1.0,NaN,1


In [386]:
# 1. Output below does not require data overrite 

final_table.to_excel('C:/Users/fanjz/Desktop/dataset/pmi/monthly_table.xlsx', index=False)
commodity_short_df_split.to_excel('C:/Users/fanjz/Desktop/dataset/pmi/commmodity_short.xlsx', index=False)
commodity_up_df_split.to_excel('C:/Users/fanjz/Desktop/dataset/pmi/commmodity_up.xlsx', index=False)
commodity_down_df_split.to_excel('C:/Users/fanjz/Desktop/dataset/pmi/commmodity_down.xlsx', index=False)

# 2. Output below requires data overrite over the existing file 

updated_index_data.to_excel('C:/Users/fanjz/Desktop/dataset/pmi/pmi_level.xlsx', index=False)
final_comment.to_excel('C:/Users/fanjz/Desktop/dataset/pmi/comments.xlsx', index=False)
sector_ranking_final.to_excel('C:/Users/fanjz/Desktop/dataset/pmi/sector_ranking.xlsx', index=False)

In [ ]:
# # 1. Output below does not require data overrite 

# final_table.to_excel('/Users/jackyliu/Desktop/dataset/pmi/monthly_table.xlsx', index=False)
# commodity_short_df_split.to_excel('/Users/jackyliu/Desktop/dataset/pmi/commmodity_short.xlsx', index=False)
# commodity_up_df_split.to_excel('/Users/jackyliu/Desktop/dataset/pmi/commmodity_up.xlsx', index=False)
# commodity_down_df_split.to_excel('/Users/jackyliu/Desktop/dataset/pmi/commmodity_down.xlsx', index=False)

# # 2. Output below requires data overrite over the existing file 

# updated_index_data.to_excel('/Users/jackyliu/Desktop/datase/pmi/pmi_level.xlsx', index=False)
# final_comment.to_excel('/Users/jackyliu/Desktop/dataset/pmi/comments.xlsx', index=False)
# sector_ranking_final.to_excel('/Users/jackyliu/Desktop/dataset/pmi/sector_ranking.xlsx', index=False)